In [4]:
from osgeo import gdal, gdal_array
import numpy as np

# Caminho para os arquivos raster
altitude_path = '/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/32S525ZN.tif'
slope_path = '/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525SN/32S525SN.tif'

# Abrir os arquivos GeoTIFF
altitude_ds = gdal.Open(altitude_path)
slope_ds = gdal.Open(slope_path)

if altitude_ds is None or slope_ds is None:
    print("Não foi possível abrir os arquivos.")
else:
    altitude_band = altitude_ds.GetRasterBand(1)
    slope_band = slope_ds.GetRasterBand(1)
    
    altitude_data = altitude_band.ReadAsArray()
    slope_data = slope_band.ReadAsArray()

    # Identificar áreas de baixa altitude e baixa declividade
    flood_risk_areas = np.where((altitude_data < 10) & (slope_data < 5), 1, 0)
    print("Áreas de risco de enchente identificadas.")

    # Salvar o resultado em um novo arquivo GeoTIFF
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create('/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/flood_risk_areas.tif', altitude_ds.RasterXSize, altitude_ds.RasterYSize, 1, gdal.GDT_Byte)
    
    # Copiar informações de georreferenciamento e projeção do dataset original
    out_ds.SetGeoTransform(altitude_ds.GetGeoTransform())
    out_ds.SetProjection(altitude_ds.GetProjection())
    
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(flood_risk_areas)
    out_band.SetNoDataValue(0)
    out_band.FlushCache()
    out_ds = None  # Fechar o arquivo

    print("Mapa de risco de enchente salvo com informações de georreferenciamento.")

Áreas de risco de enchente identificadas.
Mapa de risco de enchente salvo com informações de georreferenciamento.


ERROR 1: Key Unknown-0 of TIFFTagLocation=0 has count=0, whereas only 1 is legal.
ERROR 1: Key Unknown-0 of TIFFTagLocation=0 has count=0, whereas only 1 is legal.
Warning 1: /home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/32S525ZN.tif: GeoTIFF tags apparently corrupt, they are being ignored.


In [11]:
from osgeo import gdal
import numpy as np

url_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas/"
# Caminhos dos arquivos raster
file_paths = {
    'altitude': url_path + '32S525ZN/32S525ZN.tif',
    'slope':  url_path +'32S525SN/32S525SN.tif',
    'aspect':  url_path +'32S525ON/32S525ON.tif',
    'relief':  url_path +'32S525RS/32S525RS.tif',
    'landform':  url_path +'32S525FT/32S525FT.tif',
    'dividers':  url_path +'32S525DD/32S525DD.tif',
    'vertical_curvature':  url_path +'32S525VN/32S525VN.tif',
    'horizontal_curvature':  url_path +'32S525HN/32S525HN.tif'
}

# Função para abrir e ler dados de uma camada raster
def read_raster(file_path):
    ds = gdal.Open(file_path)
    if ds is None:
        print(f"Não foi possível abrir o arquivo: {file_path}")
        return None
    band = ds.GetRasterBand(1)
    data = band.ReadAsArray()
    return data

# Carregar todos os dados
raster_data = {key: read_raster(path) for key, path in file_paths.items()}
# Combina todos os arrays em uma única matriz de features
# Apenas pixels que não são NaN em todos os rasters serão incluídos
valid_mask = np.logical_and.reduce([data is not None for data in raster_data.values()])
if np.any(valid_mask):
    feature_matrix = np.stack([data[valid_mask] for data in raster_data.values() if data is not None], axis=-1)
else:
    print("Não há dados válidos para processar.")
# Definir pesos para cada característica com base em seu impacto no risco de alagamento
weights = np.array([0.5, -0.3, 0.1, 0.2, 0.15, -0.1, 0.25, 0.25])

# Calcular um índice de risco ponderado
risk_index = np.dot(feature_matrix, weights)
risk_index_normalized = (risk_index - risk_index.min()) / (risk_index.max() - risk_index.min())

# Reshape back to the original masked shape and create a full-sized output array
risk_index_full = np.full_like(list(raster_data.values())[0], np.nan, dtype=np.float32)
risk_index_full[valid_mask] = risk_index_normalized

# Save the risk index as a new GeoTIFF file
driver = gdal.GetDriverByName('GTiff')
output_path = url_path + "risk_index.tif"
out_ds = driver.Create(output_path, risk_index_full.shape[1], risk_index_full.shape[0], 1, gdal.GDT_Float32)
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(risk_index_full)
out_band.SetNoDataValue(np.nan)
out_band.FlushCache()
out_ds = None  # Close the file
print("Índice de risco de enchente salvo em:", output_path)


Índice de risco de enchente salvo em: /home/dv/Labs/notebook/sentinel-cloud/zonas/risk_index.tif


In [6]:
# Exemplo de pesos - esses podem precisar ser ajustados com base em conhecimento especializado ou análise estatística
weights = {
    'altitude': -1.0,  # altitudes menores aumentam o risco
    'slope': -0.5,     # declividades menores aumentam o risco
    'aspect': 0.1,     # dependendo da orientação, pode haver um pequeno impacto
    'relief': 0.3,     # maior relevo pode indicar maior escoamento e menor risco
    'landform': 0.2,
    'dividers': 0.1,
    'vertical_curvature': 0.3,
    'horizontal_curvature': 0.3
}


In [7]:
# Calcular índice de risco composto
risk_index = np.zeros_like(list(raster_data.values())[0])  # Assume que todos os rasters têm as mesmas dimensões
for key, data in raster_data.items():
    if data is not None:
        risk_index += weights[key] * data

# Normalizar o índice de risco para ficar entre 0 e 1 (opcional)
risk_index -= risk_index.min()
risk_index /= risk_index.max()


In [9]:
driver = gdal.GetDriverByName('GTiff')
out_ds = driver.Create('/home/dv/Labs/notebook/sentinel-cloud/zonas/risk_index.tif', risk_index.shape[1], risk_index.shape[0], 1, gdal.GDT_Float32)

out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(risk_index)
out_band.FlushCache()
out_ds = None  # Fechar o arquivo
print("Índice de risco de enchente salvo.")


Índice de risco de enchente salvo.


In [23]:
from osgeo import gdal
import numpy as np

# Carregar dados de altitude
altitude_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/32S525ZN.tif"
altitude_ds = gdal.Open(altitude_path)
altitude_band = altitude_ds.GetRasterBand(1)
altitude_data = altitude_band.ReadAsArray()

# Definir um limiar de altitude para áreas aquáticas em Rio Grande
water_threshold = 6,5  # Altitude em metros

# Criar uma máscara para áreas não aquáticas
land_mask = altitude_data > water_threshold

# Aplicar essa máscara ao seu raster de índice de risco
risk_index_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/risk_index.tif"
risk_ds = gdal.Open(risk_index_path)

risk_band = risk_ds.GetRasterBand(1)
risk_data = risk_band.ReadAsArray()

# Aplicar máscara
risk_data_masked = np.where(land_mask, risk_data, np.nan)  # Atribui NaN às áreas de água

# Salvar o resultado em um novo arquivo GeoTIFF
driver = gdal.GetDriverByName('GTiff')
output_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/risk_index_masked.tif"
out_ds = driver.Create(output_path, risk_ds.RasterXSize, risk_ds.RasterYSize, 1, gdal.GDT_Float32)
out_ds.SetGeoTransform(altitude_ds.GetGeoTransform())
out_ds.SetProjection(altitude_ds.GetProjection())
out_band = out_ds.GetRasterBand(1)
out_band.WriteArray(risk_data_masked)
out_band.SetNoDataValue(np.nan)
out_band.FlushCache()
out_ds = None  # Fechar o arquivo
print("Índice de risco de enchente salvo em:", output_path)


Índice de risco de enchente salvo em: /home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/risk_index_masked.tif


ERROR 1: Key Unknown-0 of TIFFTagLocation=0 has count=0, whereas only 1 is legal.
ERROR 1: Key Unknown-0 of TIFFTagLocation=0 has count=0, whereas only 1 is legal.
Warning 1: /home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/32S525ZN.tif: GeoTIFF tags apparently corrupt, they are being ignored.


In [28]:
    from osgeo import gdal
    import numpy as np
    import cv2
    
    # Carregar dados de altitude
    altitude_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/32S525ZN.tif"
    altitude_ds = gdal.Open(altitude_path)
    altitude_band = altitude_ds.GetRasterBand(1)
    altitude_data = altitude_band.ReadAsArray()
    
    # Definir um limiar de altitude para áreas aquáticas em Rio Grande
    water_threshold = 6.5  # Altitude em metros
    
    # Criar uma máscara para áreas não aquáticas
    land_mask = altitude_data > water_threshold
    
    # Aplicar essa máscara ao seu raster de índice de risco
    risk_index_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/risk_index.tif"
    risk_ds = gdal.Open(risk_index_path)
    
    risk_band = risk_ds.GetRasterBand(1)
    risk_data = risk_band.ReadAsArray()
    
    # Aplicar máscara
    risk_data_masked = np.where(land_mask, risk_data, np.nan)  # Atribui NaN às áreas de água
    
    # Salvar o resultado em um novo arquivo GeoTIFF
    driver = gdal.GetDriverByName('GTiff')
    output_path = "/home/dv/Labs/notebook/sentinel-cloud/data-out/risk_index_masked.tif"
    out_ds = driver.Create(output_path, risk_ds.RasterXSize, risk_ds.RasterYSize, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(altitude_ds.GetGeoTransform())
    out_ds.SetProjection(altitude_ds.GetProjection())
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(risk_data_masked)
    out_band.SetNoDataValue(np.nan)
    out_band.FlushCache()
    out_ds = None  # Fechar o arquivo
    print("Índice de risco de enchente salvo em:", output_path)
    
    # Gerar contornos na região de interesse
    contours, _ = cv2.findContours((land_mask * 255).astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Desenhar os contornos no arquivo de altitude
    altitude_contours = cv2.drawContours(altitude_data.astype(np.uint8), contours, -1, (255), 1)
    
    # Salvar o resultado em um novo arquivo GeoTIFF
    output_contours_path = "/home/dv/Labs/notebook/sentinel-cloud/data-out/altitude_contours.tif"
    out_contours_ds = driver.Create(output_contours_path, altitude_ds.RasterXSize, altitude_ds.RasterYSize, 1, gdal.GDT_Byte)
    out_contours_ds.SetGeoTransform(altitude_ds.GetGeoTransform())
    out_contours_ds.SetProjection(altitude_ds.GetProjection())
    out_contours_band = out_contours_ds.GetRasterBand(1)
    out_contours_band.WriteArray(altitude_contours)
    out_contours_band.FlushCache()
    out_contours_ds = None  # Fechar o arquivo
    print("Contornos da região de interesse salvos em:", output_contours_path)


ERROR 1: Key Unknown-0 of TIFFTagLocation=0 has count=0, whereas only 1 is legal.
ERROR 1: Key Unknown-0 of TIFFTagLocation=0 has count=0, whereas only 1 is legal.
Warning 1: /home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/32S525ZN.tif: GeoTIFF tags apparently corrupt, they are being ignored.


Índice de risco de enchente salvo em: /home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/risk_index_masked.tif
Contornos da região de interesse salvos em: /home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/altitude_contours.tif


In [26]:
!pip install opencv-python


  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.2 MB)


In [6]:
from osgeo import gdal
import numpy as np
import geopandas as gpd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from skimage import exposure

# Carregar dados de altitude
altitude_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/32S525ZN.tif"
ds = gdal.Open(altitude_path)
band = ds.GetRasterBand(1)
altitude_data = band.ReadAsArray()

# Normalizar e equalizar os dados de altitude para melhorar a distribuição
altitude_data_normalized = exposure.equalize_hist(altitude_data)

# Simulando dados de outras fontes (ex: umidade, precipitação) para o exemplo
# Estes dados devem ser substituídos por dados reais
humidity_data = np.random.random(altitude_data.shape)
precipitation_data = np.random.random(altitude_data.shape)

# Empilhar os rasters para formar um conjunto de características
features = np.stack([altitude_data_normalized, humidity_data, precipitation_data], axis=-1)
features = features.reshape(-1, 3)  # Reshape para (n_samples, n_features)

# Simular uma máscara de risco de inundação para treinamento
risk_labels = (altitude_data < 5).reshape(-1)  # Esta é uma simplificação, substituir por dados reais

# Dividir dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(features, risk_labels, test_size=0.2, random_state=42)

# Modelo de classificação
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prever o risco de enchente
predicted_risk = model.predict(features).reshape(altitude_data.shape)

# Converter o array para um GeoDataFrame
transform = ds.GetGeoTransform()
pixelWidth = transform[1]
pixelHeight = -transform[5]
cols = ds.RasterXSize
rows = ds.RasterYSize
xLeft = transform[0]
yTop = transform[3]

x_right = xLeft + (cols * pixelWidth)
y_bottom = yTop + (rows * pixelHeight)

x = np.linspace(xLeft, x_right, cols, endpoint=False)
y = np.linspace(yTop, y_bottom, rows, endpoint=False)
x, y = np.meshgrid(x, y)



ERROR 1: Key Unknown-0 of TIFFTagLocation=0 has count=0, whereas only 1 is legal.


In [ ]:
gdf = gpd.GeoDataFrame({'risk': predicted_risk.flatten()}, geometry=gpd.points_from_xy(x.flatten(), y.flatten()))

# Exportar para Shapefile
output_shapefile = "/home/dv/Labs/notebook/sentinel-cloud/data-out/flood_risk_zones.shp"
gdf.to_file(output_shapefile)

print(f"Shapefile de zonas de risco de enchente salvo em: {output_shapefile}")


In [5]:
from osgeo import gdal
import numpy as np
import geopandas as gpd
import cupy as cp
from cuml.ensemble import RandomForestClassifier
from cuml.model_selection import train_test_split
from skimage import exposure

# Carregar dados de altitude
altitude_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas/32S525ZN/32S525ZN.tif"
ds = gdal.Open(altitude_path)
band = ds.GetRasterBand(1)
altitude_data = band.ReadAsArray()

# Normalizar e equalizar os dados de altitude para melhorar a distribuição
altitude_data_normalized = exposure.equalize_hist(altitude_data)

# Simulando dados de outras fontes (ex: umidade, precipitação)
humidity_data = cp.random.random(altitude_data.shape)
precipitation_data = cp.random.random(altitude_data.shape)

# Empilhar os rasters para formar um conjunto de características
features = cp.stack([cp.asarray(altitude_data_normalized), humidity_data, precipitation_data], axis=-1)
features = features.reshape(-1, 3)  # Reshape para (n_samples, n_features)

# Simular uma máscara de risco de inundação para treinamento
risk_labels = cp.asarray((altitude_data < 5).reshape(-1))  # Esta é uma simplificação

# Dividir dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(features, risk_labels, test_size=0.2, random_state=42)

# Modelo de classificação
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prever o risco de enchente
predicted_risk = model.predict(features).get().reshape(altitude_data.shape)

# Aqui você continuaria com o GeoPandas para a exportação, que é executado na CPU
transform = ds.GetGeoTransform()
pixelWidth = transform[1]
pixelHeight = -transform[5]
cols = ds.RasterXSize
rows = ds.RasterYSize
xLeft = transform[0]
yTop = transform[3]

x_right = xLeft + (cols * pixelWidth)
y_bottom = yTop + (rows * pixelHeight)

x = np.linspace(xLeft, x_right, cols, endpoint=False)
y = np.linspace(yTop, y_bottom, rows, endpoint=False)
x, y = np.meshgrid(x, y)

gdf = gpd.GeoDataFrame({'risk': predicted_risk.flatten()}, geometry=gpd.points_from_xy(x.flatten(), y.flatten()))

# Exportar para Shapefile
output_shapefile = "/home/dv/Labs/notebook/sentinel-cloud/data-out/flood_risk_zones.shp"
gdf.to_file(output_shapefile)

print(f"Shapefile de zonas de risco de enchente salvo em: {output_shapefile}")

ModuleNotFoundError: No module named 'cupy'

In [11]:
import logging
def process_zone(zone, base_path, output_base_path, water_threshold):
    """
    Processa os dados geográficos para uma zona específica.
    
    Args:
        zone (str): Código da zona (ex: 'ZN', 'SN').
        base_path (str): Caminho base onde os arquivos estão armazenados.
        output_base_path (str): Caminho base para salvar os resultados.
        water_threshold (float): Limiar de altitude para áreas aquáticas.
    """
    # Construir os caminhos de entrada e saída com base na zona
    altitude_path = f"{base_path}/{zone}/{zone}.tif"
    risk_index_path = f"/home/dv/Labs/notebook/sentinel-cloud/data-utils/risk_index.tif"
    output_path = f"{output_base_path}/{zone}_risk_index_masked.tif"
    output_contours_path = f"{output_base_path}/{zone}_altitude_contours.tif"
    
    try:
        # Carregar dados de altitude
        altitude_ds = load_raster_data(altitude_path)
        altitude_data = altitude_ds.GetRasterBand(1).ReadAsArray()
        
        # Aplicar máscara de terra
        land_mask = apply_land_mask(altitude_data, water_threshold)
        
        # Carregar e aplicar máscara ao índice de risco
        risk_ds = load_raster_data(risk_index_path)
        risk_data = risk_ds.GetRasterBand(1).ReadAsArray()
        risk_data.SetGeoTransform(altitude_ds.GetGeoTransform())
        risk_data.SetProjection(altitude_ds.GetProjection())
        risk_data_masked = np.where(land_mask, risk_data, np.nan)
        
        # Salvar dados de risco processados
        save_raster_data(output_path, risk_ds, risk_data_masked, np.nan)
        
        # Gerar e salvar contornos
        save_contours(altitude_ds, land_mask, output_contours_path)
        
        logging.info(f"Processamento concluído para a zona: {zone}")
    except Exception as e:
        logging.error(f"Erro ao processar a zona {zone}: {e}")

# Configurações e chamada da função
base_path = "/home/dv/Labs/notebook/sentinel-cloud/zonas"
output_base_path = "/home/dv/Labs/notebook/sentinel-cloud/data-out"
water_threshold = 6.5

zones = ['32S525ZN', '32S525SN', '32S525ON', '32S525RS', '32S525FT', '32S525DD', '32S525VN', '32S525HN']
for zone in zones:
    process_zone(zone, base_path, output_base_path, water_threshold)


ERROR:root:Erro ao processar a zona 32S525ZN: name 'load_raster_data' is not defined
ERROR:root:Erro ao processar a zona 32S525SN: name 'load_raster_data' is not defined
ERROR:root:Erro ao processar a zona 32S525ON: name 'load_raster_data' is not defined
ERROR:root:Erro ao processar a zona 32S525RS: name 'load_raster_data' is not defined
ERROR:root:Erro ao processar a zona 32S525FT: name 'load_raster_data' is not defined
ERROR:root:Erro ao processar a zona 32S525DD: name 'load_raster_data' is not defined
ERROR:root:Erro ao processar a zona 32S525VN: name 'load_raster_data' is not defined
ERROR:root:Erro ao processar a zona 32S525HN: name 'load_raster_data' is not defined


In [14]:
    out_ds.SetGeoTransform(source_ds.GetGeoTransform())  # Configura a transformação geográfica
    out_ds.SetProjection(source_ds.GetProjection())  # 

ERROR:root:Erro ao processar a zona 32S525ZN: 'numpy.ndarray' object has no attribute 'SetGeoTransform'
ERROR:root:Erro ao processar a zona 32S525SN: 'numpy.ndarray' object has no attribute 'SetGeoTransform'
ERROR:root:Erro ao processar a zona 32S525ON: 'numpy.ndarray' object has no attribute 'SetGeoTransform'
ERROR:root:Erro ao processar a zona 32S525RS: 'numpy.ndarray' object has no attribute 'SetGeoTransform'
ERROR:root:Erro ao processar a zona 32S525FT: 'numpy.ndarray' object has no attribute 'SetGeoTransform'
ERROR:root:Erro ao processar a zona 32S525DD: 'numpy.ndarray' object has no attribute 'SetGeoTransform'
ERROR:root:Erro ao processar a zona 32S525VN: 'numpy.ndarray' object has no attribute 'SetGeoTransform'
ERROR:root:Erro ao processar a zona 32S525HN: 'numpy.ndarray' object has no attribute 'SetGeoTransform'
